In [2]:
import torch
from torch import nn
from torch.nn import init

net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1))  # pytorch已进行默认初始化

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)
tensor(0.2772, grad_fn=<SumBackward0>)


# 一、访问模型参数

In [6]:
print(type(net.named_parameters))
for name,param in net.named_parameters():
    print(name,param.size())

<class 'method'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


## 1.访问任意一层的参数
torch.nn.parameter.Parameter是Tensor的子类，

In [8]:
for name,param in net[0].named_parameters():
    print(name,param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


如果一个Tensor是Parameter,那么会自动添加到模型的参数列表里。

In [11]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20, 20)
    def forward(self, x):
        pass

n = MyModel()
for name, param in n.named_parameters():
    print(name)
# weight2没有被添加到网络的参数列表中

weight1


## 访问参数的[参数值]和[梯度]
因为Parameter是Tensor，所以具有data，grad属性。
用来访问参数的值，以及参数的梯度

In [14]:
weight_0 = list(net[0].parameters())[0]
print('参数值：',weight_0.data)
Y.backward()
print('梯度值',weight_0.grad)

参数值： tensor([[ 0.1340,  0.0383,  0.2253, -0.3725],
        [ 0.2172, -0.1447, -0.3610, -0.4326],
        [-0.2553, -0.1580, -0.3386,  0.1231]])
梯度值 tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.2053, -0.1221, -0.1945, -0.0116],
        [ 0.0463,  0.0928,  0.0848,  0.1219]])


# 二、初始化模型参数
init模块

## 一般初始化方法

In [19]:
# 随机初始化权重参数
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean = 0, std = 0.01)
        print(name, param.data)

0.weight tensor([[-0.0182,  0.0035,  0.0063,  0.0009],
        [-0.0120,  0.0047, -0.0080,  0.0160],
        [ 0.0125, -0.0148,  0.0165, -0.0010]])
2.weight tensor([[0.0069, 0.0052, 0.0094]])


In [20]:
# 使用常数来初始化权重参数
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param,val=0)
        print(name, param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


## 自定义初始化方法

In [22]:
# 自定义一个初始化方法，
# Parameter是Tensor，自定义方法传入tensor即可
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float()

for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)


0.weight tensor([[-0.0000,  0.0000, -0.0000, -0.0000],
        [ 0.0000, -0.0000,  7.8551, -9.2843],
        [-9.9752, -0.0000, -5.1001, -0.0000]])
2.weight tensor([[ 0.0000, -7.9318, -0.0000]])


# 共享模型参数

In [79]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear) # 使用同一个Module实例初始化net 
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

print(net[0] == net[1]) # 两个全连接层是等价的

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])
True


**因为模型参数里包含了梯度，所以在反向传播计算时，这些共享的参数的梯度是累加的:**

In [80]:
x = torch.ones(1,1)
y = net(x).sum()
print(y)
y.backward()
print(net[0].weight.grad)# 单次梯度是3，两次所以就是6

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
